<ul>
<li>Provide a list of audiences activated each quarter, broken down by vehicle.</li>
<li>Include details about the activation path, such as direct vs. NS vs. LR, and the use of Permutive.</li>
<li>Describe the nature of the audience, e.g., whether it's 3p (third-party) or 1p (first-party), etc.</li>
<li>Additionally, please specify the associated media spend for each audience.</li>
<li>Include information about the funnel stage, including segments like high propensity, active, etc.</li>
<li>Present performance results, with a preference for comparing changes to baseline data if it is available.</li>
<ul>

This analysis uses the output of the "Delve File Manipulation"

<h2>Library</h2>

In [1]:
import pandas as pd
import warnings

warnings.filterwarnings('ignore')

<h2>Function</h2>

<h2>Main</h2>

In [2]:
folder_path = r'C:\Users\Rafael_Fagundes\Downloads\delve_compiled_data.csv'

df = pd.read_csv(folder_path)

In [3]:
df = df[(df['Segment ID'] != 0) & (df['Segment ID'] != '0')]

In [4]:
df_audiences = df[['Country', 'Fiscal Quarter', 'BU', 'Audience Type', 'Segment ID', 'Spend', 'Net Rev']]

df_audiences = df_audiences.groupby(['Segment ID', 'Country', 'BU' ,'Audience Type', 'Fiscal Quarter'])[['Spend', 'Net Rev']].sum().reset_index()

In [5]:
# Create a new column 'Status' to initially mark all segments as 'New'
df_audiences['Status'] = 'New'

# Iterate through each unique 'Segment ID'
for audience_id in df_audiences['Segment ID'].unique():
    # Find the rows where 'Segment ID' matches the current ID
    segment_rows = df_audiences[df_audiences['Segment ID'] == audience_id]
    
    # Check if there is more than one row for this segment (i.e., it was reused)
    if len(segment_rows) > 1:
        # Set 'Status' to 'Reused' for all rows except the first one
        df_audiences.loc[segment_rows.index[1:], 'Status'] = 'Reused'

In [6]:
# Pivot table 1: Counting 'Segment ID'
pivot_df_count = df_audiences.pivot_table(
    index=['Country', 'BU', 'Audience Type', 'Status'],
    columns='Fiscal Quarter',
    values='Segment ID',
    aggfunc='count',
    margins=True,  # Add a total line
    margins_name='Total'  # Name for the total line
)

# Pivot table 2: Summing 'Spend' and 'Net Rev'
pivot_df_sum = df_audiences.pivot_table(
    index=['Country', 'BU', 'Audience Type', 'Status'],
    columns='Fiscal Quarter',
    values=['Spend', 'Net Rev'],
    aggfunc='sum',
    margins=True,  # Add a total line
    margins_name='Total'  # Name for the total line
)

# Format the pivot table cells as integers and fill NaN values with an empty string for both pivot tables
pivot_df_count = pivot_df_count.applymap(lambda x: f'{int(x):,}' if not pd.isna(x) else '')
pivot_df_sum = pivot_df_sum.applymap(lambda x: f'{int(x):,}' if not pd.isna(x) else '')

# Merge the two pivot tables
merged_pivot = pd.concat([pivot_df_count, pivot_df_sum], axis=1)

# Optionally, reset the index if needed
# merged_pivot.reset_index(inplace=True)

merged_pivot


2023-Q4 2024-Q1 2024-Q2 2024-Q3  \
Country       BU  Audience Type Status                                   
Australia     B2B 1PD           New                                      
                  3PD           New          4                       5   
                                Reused               4       4       4   
              CSB 1PD           New                                  3   
                                Reused                                   
Canada        B2B 3PD           New                          7           
                                Reused                               4   
              CSB 1PD           New                  6               4   
                                Reused                       6       6   
                  3PD           New                 18      11           
                                Reused                      18      15   
France        CSB 1PD           New                                  1   
                                Reused                                   
India         B2B 1PD           Reused                                   
                  3PD           New          4                       4   
                                Reused               4       4       4   
Japan         B2B 1PD           Reused                                   
                  3PD           New          4                       4   
                                Reused               4       4       4   
Malaysia      B2B 3PD           New          1       1               2   
                                Reused               1       2       2   
Singapore     B2B 3PD           New          2                       2   
                                Reused               4       4       6   
United States B2B 1PD           New                  8       5       8   
                                Reused                       8      13   
                  3PD           New                 25      21      13   
                                Reused                      25      46   
              CSB 1PD           New                 28       1       5   
                                Reused                      27      27   
                  3PD           New          3      17       8       3   
                                Reused               5      21      25   
Total                                       18     125     176     210   

                                       2024-Q4 Total (Net Rev, 2023-Q4)  \
Country       BU  Audience Type Status                                    
Australia     B2B 1PD           New          3     3                      
                  3PD           New          1    10                  0   
                                Reused       9    21                      
              CSB 1PD           New          9    12                      
                                Reused       2     2                      
Canada        B2B 3PD           New          5    12                      
                                Reused       6    10                      
              CSB 1PD           New               10                      
                                Reused      10    22                      
                  3PD           New               29                      
                                Reused      11    44                      
France        CSB 1PD           New                1                      
                                Reused       1     1                      
India         B2B 1PD           Reused       3     3                      
                  3PD           New                8                  0   
                                Reused       8    20                      
Japan         B2B 1PD           Reused       3     3                      
                  3PD           New          1     9                  0   
                      

In [7]:
df_audiences.to_csv('csv/df_audiences.csv')